# **Space X  Falcon 9 First Stage Landing Prediction**

In [2]:
import sys
import requests
import unicodedata

from bs4 import BeautifulSoup

import re          # regular expressions

import pandas as pd

In [3]:
# VAMOS A SACAR LOS DATOS DE 

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


In [4]:
# FUNCIONES QUE VAMOS A UTILIZAR PARA HACER EL WEB SCRAPPING

def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]
    #   .strip() elimina espacios en blanco antes y después de strings
    #   la función itera los dos primeros elementos de data_time (limpiados con .strip()) en la lista 'table_cells.strings'

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):                        # elimina el cambio de pauta (\n en python)
        row.br.extract()
    if row.a:                           # elimina tsangrías
        row.a.extract()
    if row.sup:                         # elimina negritas
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)   # column_name es la union de los contenidos restantes
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


### TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [5]:
response = requests.get(static_url)
response.status_code

200

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
soup.title

# ASÍ SE COMPRUEBA QUE LO HAYA LEÍDO BIEN

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


In [8]:
html_tables = soup.find_all('table')

In [9]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [10]:
# VAMOS A CONSEGUIR LOS NOMBRES DE LAS COLUMNAS

column_names = []

th_items = first_launch_table.find_all('th')   # 'th' = 'table header'


for th in th_items:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

column_names


['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

## TASK 3: Create a data frame by parsing the launch HTML tables

In [11]:
# CREAMOS UN DICCIONARIO QUE CONVERTIREMOS EN DF

launch_dict = dict.fromkeys(column_names)

launch_dict

{'Flight No.': None,
 'Date and time ( )': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None}

In [12]:
# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [13]:
tablas = soup.find_all('table',"wikitable plainrowheaders collapsible")
len(tablas)

9

In [14]:
# HAY QUE LLENAR LAS LISTAS

extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):   #para cada tabla
   # get table row 
    for rows in table.find_all("tr"):                                                                 # para cada fila de la tabla ('tr')
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        print(flag)
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            launch_dict={}
            # Flight Number value
            launch_dict['Flight No.'] = flight_number
            
            datatimelist=date_time(row[0])
            
            # Date value
            
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date
            
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'] = time
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            launch_dict['Version Booster'] = bv
            
            if not(bv):
                bv=row[1].a.string
            
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch Site'] = launch_site
            
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = payload
            
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = payload_mass
            
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = orbit
            
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            try:
                customer = row[6].a.string
            except:
                customer = 'Various'

            launch_dict['Customer'] = customer
            
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_outcome 
            
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'] = booster_landing
            
            #print(booster_landing)
            

False
True
False
True
False
True
False
True
False
False
True
False
True
False
True
False
False
True
False
True
False
True
False
True
False
True
False
True
False
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
False
False
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
False
True
False
True
False
False
False
False
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
False
True
False
True
False
True
False
False
False
False
False
True
False
True
False
True
False
False
False
False
False
True
False
True
False
True
False
True
False
True
False
False
True
False
True

In [15]:
launch_dict

{'Flight No.': '121',
 'Date': '6 June 2021',
 'Time': '04:26',
 'Version Booster': 'F9 B5',
 'Launch Site': 'CCSFS',
 'Payload': 'SXM-8',
 'Payload mass': '7,000 kg',
 'Orbit': 'GTO',
 'Customer': 'Sirius XM',
 'Launch outcome': 'Success\n',
 'Booster landing': 'Success'}

In [16]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [17]:
df.head()

,Flight No.,Date,Time,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,121,6 June 2021,04:26,F9 B5,CCSFS,SXM-8,"7,000 kg",GTO,Sirius XM,Success\n,Success


In [18]:
df.shape

(1, 11)

-------------------
---------------------
---------------------

# MÉTODO JA

In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from lxml import html
import requests
# VAMOS A SACAR LOS DATOS DE 
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


# Realiza la solicitud HTTP GET para obtener el contenido de la página
response = requests.get(url)

tree = html.fromstring(response.content)

# Encuentra la tabla utilizando el XPath proporcionado
table_xpath = '//*[@id="mw-content-text"]/div[1]/table[2]'

table_element = tree.xpath(table_xpath)

df = pd.read_html(html.tostring(table_element[0]))[0]
df = df[df.index % 2 == 0]
df.set_index('Flight No.', inplace=True)

df.head()

,Date and time (UTC),"Version, Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launch outcome,Booster landing
Flight No.,,,,,,,,,
8,"6 January 2014, 22:06[41]",F9 v1.1,"CCAFS, SLC-40",Thaicom 6[22],"3,325 kg (7,330 lb)",GTO,Thaicom,Success[42],No attempt [43]
9,"18 April 2014, 19:25[21]",F9 v1.1,"Cape Canaveral, LC-40",SpaceX CRS-3[22] (Dragon C105),"2,296 kg (5,062 lb)[45]",LEO (ISS),NASA (CRS),Success,Controlled (ocean) [d][46]
10,"14 July 2014, 15:15",F9 v1.1,"Cape Canaveral, LC-40",Orbcomm-OG2-1 (6 satellites)[22],"1,316 kg (2,901 lb)",LEO,Orbcomm,Success[51],Controlled (ocean)[d][46]
11,"5 August 2014, 08:00",F9 v1.1,"Cape Canaveral, LC-40",AsiaSat 8[22][54][55],"4,535 kg (9,998 lb)",GTO,AsiaSat,Success[56],No attempt [57]
12,"7 September 2014, 05:00",F9 v1.1 B1011[8],"Cape Canaveral, LC-40",AsiaSat 6[22][54][58],"4,428 kg (9,762 lb)",GTO,AsiaSat,Success[59],No attempt


-------------------
--------------------
-------------------

# PARA TODAS LAS TABLAS

In [35]:
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
import requests

# URL de la página
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Obtener el contenido HTML de la página
response = requests.get(url)

# Parsear el contenido HTML
tree = html.fromstring(response.content)

# Crear un objeto BeautifulSoup para trabajar con el HTML
soup = BeautifulSoup(response.text, 'html.parser')

numb = range(0, 10)
table_xpath = '//*[@id="mw-content-text"]/div[1]/table[2]'
xpath_list = []

dfs = []

# Generar los XPath modificados
for n in numb:
    modified_xpath = table_xpath.replace('2', str(n))
    xpath_list.append(modified_xpath)

# Leer y filtrar las tablas
for xpath in xpath_list:
    table_elements = tree.xpath(xpath)
    if table_elements:
        # Leer la tabla
        df = pd.read_html(html.tostring(table_elements[0]))[0]
        # Eliminar las filas duplicadas en la columna 'Flight No.'
        df = df[~df.duplicated(subset='Flight No.', keep='first')]
        dfs.append(df)

# Concatenar los DataFrames en un único DataFrame
df = pd.concat(dfs, ignore_index=True)

# Establecer la columna 'Flight No.' como el índice
df.set_index('Flight No.', inplace=True)

# Mostrar el DataFrame resultante
print(df)


                   Date and time (UTC)   Version, Booster [b]  \
Flight No.                                                      
1                   4 June 2010, 18:45  F9 v1.0[7] B0003.1[8]   
2           8 December 2010, 15:43[13]  F9 v1.0[7] B0004.1[8]   
3               22 May 2012, 07:44[17]  F9 v1.0[7] B0005.1[8]   
4            8 October 2012, 00:35[21]  F9 v1.0[7] B0006.1[8]   
5                  1 March 2013, 15:10  F9 v1.0[7] B0007.1[8]   
...                                ...                    ...   
117              9 May 2021 06:42[656]                    NaN   
118             15 May 2021 22:56[659]                    NaN   
119             26 May 2021 18:59[664]                    NaN   
120             3 June 2021 17:29[667]                    NaN   
121             6 June 2021 04:26[673]                    NaN   

                   Launch site  \
Flight No.                       
1                CCAFS, SLC-40   
2                CCAFS, SLC-40   
3                C